# Statistical Hypothesis Testing II

In this notebook, we will implement and apply **statistical hypothesis tests** to make inferences about risks of learning algorithms.

At the start, we will compare two learning algorithms on one domain via the paired $t$-test.

Subsequently, we will compare the two learning algorithm across multiple domains via the Wilcoxon signed-rank test.

### **Table of Contents**
1. [Paired $t$-test](#paired-t-test)
2. [Wilcoxon Signed-rank Test](#wilcoxon-signed-rank-test)

In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np

from scipy import stats

### **1. Paired $t$-test** <a class="anchor" id="paired-t-test"></a>

We implement the function [`t_test_paired`](../e2ml/evaluation/_paired_tests.py) in the [`e2ml.evaluation`](../e2ml/evaluation) subpackage. Once, the implementation has been completed, we check it for varying types of tests.

In [2]:
from e2ml.evaluation import t_test_paired
r_1 = np.round(stats.norm.rvs(loc=0.1, scale=0.03, size=40, random_state=0), 2)
r_2 = np.round(stats.norm.rvs(loc=0.12, scale=0.03, size=40, random_state=1), 2)
mu_0 = 0
t_statistic, p = t_test_paired(sample_data_1=r_1, sample_data_2=r_2, mu_0=mu_0, test_type="right-tail")
assert np.round(t_statistic, 4) == -1.4731 , 'The paired t-test statistic must be ca. -1.4731.' 
assert np.round(p, 4) == 0.9256, 'The p-value must be ca. 0.9256 for the one-sided right-tail test.' 
t_statistic, p = t_test_paired(sample_data_1=r_1, sample_data_2=r_2, mu_0=mu_0, test_type="left-tail")
assert np.round(t_statistic, 4) == -1.4731 , 'The paired t-test statistic must be ca. -1.4731.' 
assert np.round(p, 4) == 0.0744, 'The p-value must be ca. 0.0744 for the one-sided left-tail test.' 
t_statistic, p = t_test_paired(sample_data_1=r_1, sample_data_2=r_2, mu_0=mu_0, test_type="two-sided")
assert np.round(t_statistic, 4) == -1.4731 , 'The paired t-test statistic must be ca. -1.4731.' 
assert np.round(p, 4) == 0.1487, 'The p-value must be ca. 0.1487 for the two-sided test.' 

Next, we want to check whether a *support vector classifier* (SVC) significantly outperforms a *Gaussian process classifier* (GPC) on the data set breast cancer, where we use the zero-one loss as performance measure and the paired $t$-test with $\alpha=0.01$. Design and perform the corresponding evaluation study.

In [5]:
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.datasets import load_breast_cancer
from e2ml.evaluation import cross_validation, zero_one_loss
from e2ml.preprocessing import StandardScaler

# load data
X, y = load_breast_cancer(return_X_y=True)

# create folds
sample_indices = np.arange(len(y), dtype=int)
n_folds = 8
train, test = cross_validation(
    sample_indices=sample_indices,
    n_folds=n_folds,
    y=y,
    random_state=0
)

# result risk list
risks_gpc = []
risks_svc = []

# loop through fodls
for train_, test_ in zip(train, test):
    # scale data
    scaler = StandardScaler()
    scaler.fit(X[train_])
    X_train = scaler.transform(X[train_])
    X_test = scaler.transform(X[test_])

    # init classifiers
    gpc = GaussianProcessClassifier(random_state=0)
    svc = SVC(random_state=0)

    # train classifiers
    gpc.fit(X_train, y[train_])
    svc.fit(X_train, y[train_])

    # predict on test set
    y_gpc = gpc.predict(X[test_])
    y_svc = svc.predict(X[test_])

    # compute zero-one loss
    loss_gpc = zero_one_loss(y_true=y[test_], y_pred=y_gpc)
    loss_svc = zero_one_loss(y_true=y[test_], y_pred=y_svc)

    # save results of folds
    risks_gpc.append(loss_gpc)
    risks_svc.append(loss_svc)

# compare paired t-test to compare performances
alpha = 0.01
t_statistic, p = t_test_paired(sample_data_1=risks_gpc, sample_data_2=risks_svc, mu_0=0, test_type='right-tail')

if p <= alpha:
    print('H_0 is rejected --> SVC outperforms GPC')
else:
    print(f'Not enough evidence to reject H_0 --> cannot say whether SVC outperforms GPC ({p}) ({alpha})')

Not enough evidence to reject H_0 --> cannot say whether SVC outperforms GPC (nan) (0.01)


/Users/Aleyna/PythonProjects/e2ml/e2ml/evaluation/_one_sample_tests.py:90: RuntimeWarning: invalid value encountered in double_scalars
  t_statistic = (empirical_mean-mu_0) / np.sqrt((empirical_std**2 / sample_size))


#### **Questions:**
1. (a) What are possible issues of your conducted evaluation study?
   
   TODO
   TODO

### **2. Wilcoxon Signed-rank Test** <a class="anchor" id="wilcoxon-signed-rank-test"></a>

We implement the function [`wilcoxon_signed_rank_test`](../e2ml/evaluation/_paired_tests.py) in the [`e2ml.evaluation`](../e2ml/evaluation) subpackage. Once, the implementation has been completed, we check it for varying types of tests.

In [20]:
from e2ml.evaluation import wilcoxon_signed_rank_test

# Test for exact computation.
r_1 = stats.norm.rvs(loc=0.1, scale=0.03, size=10, random_state=0)
r_2 = stats.norm.rvs(loc=0.15, scale=0.03, size=10, random_state=1)
d = r_2 - r_1
w_statistic, p = wilcoxon_signed_rank_test(sample_data_1=d, test_type="right-tail")
assert w_statistic == 47 , 'The positive rank sum statistic must be 47.' 
assert np.round(p, 4) == 0.0244, 'The p-value must be ca. 0.0244 for the one-sided right-tail test.' 
w_statistic, p = wilcoxon_signed_rank_test(sample_data_1=d, test_type="left-tail")
assert w_statistic == 47 , 'The positive rank sum statistic must be 47.' 
assert np.round(p, 4) == 0.9814, 'The p-value must be ca. 0.9814 for the one-sided left-tail test.' 
w_statistic, p = wilcoxon_signed_rank_test(sample_data_1=d, test_type="two-sided")
assert w_statistic == 47 , 'The positive rank sum statistic must be 47.' 
assert np.round(p, 4) == 0.0488, 'The p-value must be ca. 0.0488 for the two-sided test.' 

# Test for approximative computation.
r_1 = stats.norm.rvs(loc=2, scale=0.3, size=100, random_state=0)
r_2 = stats.norm.rvs(loc=2.1, scale=0.3, size=100, random_state=1)
d = r_2 - r_1
w_statistic, p = wilcoxon_signed_rank_test(sample_data_1=d, test_type="right-tail")
assert w_statistic == 3303 , 'The positive rank sum statistic must be 3303.' 
assert np.round(p, 4) == 0.0037, 'The p-value must be ca. 0.0037 for the one-sided right-tail test.' 
w_statistic, p = wilcoxon_signed_rank_test(sample_data_1=d, test_type="left-tail")
assert w_statistic == 3303 , 'The positive rank sum statistic must be 3303.' 
assert np.round(p, 4) == 0.9963, 'The p-value must be ca. 0.9963 for the one-sided left-tail test.' 
w_statistic, p = wilcoxon_signed_rank_test(sample_data_1=d, test_type="two-sided")
assert w_statistic == 3303 , 'The positive rank sum statistic must be 3303.' 
assert np.round(p, 4) == 0.0075, 'The p-value must be ca. 0.0075 for the two-sided test.' 

Next, we want to check whether a *support vector classifier* (SVC) significantly outperforms a *Gaussian process classifier* (GPC) on ten articially generated data sets, where we use the zero-one loss as performance measure and the paired $t$-test with $\alpha=0.01$. Design and perform the corresponding evaluation study.

In [22]:
from sklearn.datasets import make_classification

# Create 10 articial data sets.
data_sets = []
n_classes_list = np.arange(2, 12)
for n_classes in n_classes_list:
    X, y = make_classification(
        n_samples=500, n_classes=n_classes, class_sep=2, n_informative=10, random_state=n_classes
    )
    data_sets.append((X, y))

# create folds
sample_indices = np.arange(len(y), dtype=int)
n_folds = len(data_sets)
train, test = cross_validation(
    sample_indices=sample_indices,
    n_folds=n_folds,
    y=y,
    random_state=0
)

# result risk list
risks_gpc = []
risks_svc = []

# loop through fodls
for train_, test_, dataset_ in zip(train, test, data_sets):
    # scale data
    X, y = dataset_
    scaler = StandardScaler()
    scaler.fit(X[train_])
    X_train = scaler.transform(X[train_])
    X_test = scaler.transform(X[test_])

    # init classifiers
    gpc = GaussianProcessClassifier(random_state=0)
    svc = SVC(random_state=0)

    # train classifiers
    gpc.fit(X_train, y[train_])
    svc.fit(X_train, y[train_])

    # predict on test set
    y_gpc = gpc.predict(X[test_])
    y_svc = svc.predict(X[test_])

    # compute zero-one loss
    loss_gpc = zero_one_loss(y_true=y[test_], y_pred=y_gpc)
    loss_svc = zero_one_loss(y_true=y[test_], y_pred=y_svc)

    # save results of folds
    risks_gpc.append(loss_gpc)
    risks_svc.append(loss_svc)

# compare paired t-test to compare performances
alpha = 0.01
w_statistic, p = wilcoxon_signed_rank_test(sample_data_1=risks_gpc, sample_data_2=risks_svc, test_type='right-tail')

if p <= alpha:
    print('H_0 is rejected --> SVC outperforms GPC')
else:
    print(f'Not enough evidence to reject H_0 --> cannot say whether SVC outperforms GPC ({p}) ({alpha})')


Not enough evidence to reject H_0 --> cannot say whether SVC outperforms GPC (1.0) (0.01)


#### **Questions:**
2. (a) What are possible issues of your conducted evaluation study?
   
   TODO
   TODO